In [4]:
import pandas as pd
import os
import torch.nn as nn

/home/ugrads/a/aa_ron_su/miniconda3/envs/clinical_t5/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
root = '/home/ugrads/a/aa_ron_su/physionet.org/files/clinical-t5/1.0.0/'
data_path = '/data/datasets/mimiciv_notes/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv'
model_path = root + 'Clinical-T5-Base/'
finetune_model_path = root + 'Clinical-T5-Base_ft_vent/'
temivef_train_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT.csv'
# temivef_test_NOTE_TARGET1_FT_path = '/home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_test_NOTE_TARGET1_FT.csv'
model_out = root + "meta_ft_classify.pt"

In [6]:
train = pd.read_csv(temivef_train_NOTE_TARGET1_FT_path)
print(f"reading notes and target from {temivef_train_NOTE_TARGET1_FT_path}")

reading notes and target from /home/ugrads/a/aa_ron_su/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT.csv


In [33]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "Clinical-T5-Base"
tokenizer = AutoTokenizer.from_pretrained("Clinical-T5-Base")
model = AutoModelForSeq2SeqLM.from_pretrained("Clinical-T5-Base")

<generator object Module.parameters at 0x7fe3c89acb30>

In [5]:
# mimic_iv_train_NOTE_EMBEDDINGS_target1 = pd.read_csv(mimic_iv_train_NOTE_EMBEDDINGS_target1_path) # this contains the embeddings and the target
# df = mimic_iv_train_NOTE_EMBEDDINGS_target1.filter(regex='emb\d{1,2}', axis=1)
# df = pd.concat([df, mimic_iv_train_NOTE_EMBEDDINGS_target1['delta_in_2_days']], axis=1)

# Use this later in pipeline.... 

In [4]:
# df = pd.read_csv(mimic_iv_train_NOTE_TARGET1_path)
# df = df[['text','delta_in_2_days']].copy()

In [5]:
# print(len(df))
# df.dropna(inplace=True)
# print(len(df))

28724
11753


In [20]:
# def tokenize_function(example): 
#     return tokenizer(example, padding="max_length", truncation=True, return_tensors = "pt")

In [13]:
# from datasets import Dataset
# df = pd.read_csv(mimic_iv_train_NOTE_path)
# df.dropna(inplace=True)
# dataset = Dataset.from_pandas(df)

NameError: name 'mimic_iv_train_NOTE_path' is not defined

In [10]:
import torch
import os 
train_tensor_filename = "train_tensor.pt"

train_tensor = None
if os.path.isfile(train_tensor_filename):
    train_tensor = torch.load(train_tensor_filename)
    print(f"tokenized note input_ids loaded from {train_tensor_filename}")
else:
    train_texts = train['text'].tolist()
    tokenized_train_notes = tokenizer(train_texts, truncation=True, padding=True, return_tensors = "pt")
    train_tensor = tokenized_train_notes.input_ids
    torch.save(train_tensor, train_tensor_filename)
    print(f"train notes tokenized and saved to {train_tensor_filename}")

tokenized note input_ids loaded from train_tensor.pt


In [11]:
train_tensor

tensor([[ 5570,    10,     3,  ...,    10,  1714,     1],
        [ 5570,    10,     3,  ...,  1678,    87,     1],
        [ 5570,    10,     3,  ...,  2841,     5,     1],
        ...,
        [ 5570,    10,     3,  ...,    23,   106,     1],
        [ 5570,    10,     3,  ...,    11, 16643,     1],
        [ 5570,    10,     3,  ...,     7,    28,     1]])

In [23]:
import numpy as np
def group_k_fold(ID, num_folds, seed=None):
    ID             = ID.astype(int)
    ID_unique_srtd = np.sort(np.unique(ID))

    ID_counts = [len(ID_unique_srtd)//num_folds] * num_folds
    for i in range(len(ID_unique_srtd)%num_folds):
        ID_counts[i]+=1

    assert sum(ID_counts)==len(ID_unique_srtd)

    fold_idxs = np.hstack([[i]*id_count for i, id_count in enumerate(ID_counts)])

    # if seed is not None:
    #     with temp_seed(seed):
    #         np.random.shuffle(fold_idxs)
    # else:
    np.random.shuffle(fold_idxs)

    gkf = []
    for fold_idx in range(num_folds):
        train_ids = ID_unique_srtd[np.where(fold_idxs!=fold_idx)[0]]
        test_ids  = ID_unique_srtd[np.where(fold_idxs==fold_idx)[0]]
        gkf.append((np.where(np.isin(ID, train_ids))[0], np.where(np.isin(ID, test_ids))[0]))
    
    return gkf

# define 5 fold cv
num_folds = 5
kfold = group_k_fold(train['ICUSTAY_ID'], num_folds)

In [32]:
[(len(kfold[i][0]), len(kfold[i][1])) for i in range(len(kfold))]

[(11669, 2904), (11668, 2905), (11635, 2938), (11660, 2913), (11660, 2913)]

In [20]:
torch.cuda.set_device(3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")

device: cuda


In [22]:
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

train_inputs = train_tensor
train_labels = torch.tensor(train['delta_in_2_days'].to_numpy())
train_dataset = TensorDataset(train_inputs.to(device), train_labels.to(device)) 

In [24]:
from torch.utils.data import Subset
batch_size = 4


for fold, (train_idx, val_idx) in enumerate(kfold):
    print(train_idx.shape, val_idx.shape)
    val_set = Subset(train_dataset, val_idx)
    val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
    train_set = Subset(train_dataset, train_idx)
    train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    print(len(data_loader))

(9723,) (4850,)
(9720,) (4853,)
(9703,) (4870,)


In [15]:
# notes_to_extract = df['text']
# texts = notes_to_extract.tolist()
# tokenized_notes_to_extract = tokenizer(texts, truncation=True, padding=True, return_tensors = "pt")

# # test_note_to_extract = notes_to_extract.iloc[0]
# # tokenized_test_note = tokenize_function(test_note_to_extract)
# # tokenized_test_note.keys()

In [23]:
# %load_ext autoreload
# %autoreload 2

In [36]:
from T5EncoderForSequenceClassification import T5EncoderForSequenceClassification, T5EncoderClassificationHead

In [37]:
from transformers import T5Config

encoder = model.get_encoder() # we only need the clinical-t5 encoder for our purposes

config = T5Config(
    hidden_size=768,
    classifier_dropout=None,
    num_labels=2,
    hidden_dropout_prob=0.01,
    last_hidden_size=64
)
classifier = T5EncoderForSequenceClassification(encoder, config)

In [43]:
classifier.named_parameters

<bound method Module.named_parameters of T5EncoderForSequenceClassification(
  (encoder): T5Stack(
    (embed_tokens): Embedding(32115, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
             

In [6]:
# import torch
# import torch.nn as nn
# from torch.nn.utils import parameters_to_vector

# num_params = len(parameters_to_vector(encoder.parameters()))

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
classifier.to(device)

In [15]:
num_params

109618560

In [16]:
len(parameters_to_vector(classifier.classifier.parameters()))

639938

In [ ]:
classifier.encoder.named_parameters

In [ ]:
classifier.classifier

T5EncoderClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (last_dense): Linear(in_features=768, out_features=64, bias=True)
  (dropout): Dropout(p=0.01, inplace=False)
  (out_proj): Linear(in_features=64, out_features=2, bias=True)
)

In [22]:
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np
import torch

# Define your training data
train_inputs = tokenized_notes_to_extract.input_ids
# train_labels = torch.tensor(np.random.rand(len(train_inputs)))
train_labels = torch.tensor(df['delta_in_2_days'].to_numpy())
train_dataset = TensorDataset(train_inputs.to(device), train_labels.to(device))

In [27]:
train_dataset[0][0].device.type

'cuda'

In [10]:
# batch_size = 32
# train_sampler = RandomSampler(train_dataset)
# train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

In [27]:
batch_size = 16
num_epochs = 1
learning_rate = 5e-5
adam_epsilon = 1e-8
max_grad_norm = 1.0

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

In [28]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.AdamW(classifier.classifier.parameters(), lr=learning_rate, eps=adam_epsilon)

In [29]:
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [30]:
# freeze encoder weights:
for param in classifier.encoder.parameters():
    param.requires_grad = False

In [31]:
from tqdm import tqdm

classifier.classifier.train()
for epoch in tqdm(range(num_epochs)):
    for step, batch in enumerate(train_dataloader):
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = classifier.forward(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(classifier.classifier.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        # Print progress every 10 steps
        if step % 10 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs} | Step {step}/{len(train_dataloader)} | Loss {loss.item():.4f}")

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1 | Step 0/735 | Loss 0.6700
Epoch 1/1 | Step 10/735 | Loss 1.0951
Epoch 1/1 | Step 20/735 | Loss 0.3144
Epoch 1/1 | Step 30/735 | Loss 0.1830
Epoch 1/1 | Step 40/735 | Loss 0.1017
Epoch 1/1 | Step 50/735 | Loss 0.0713
Epoch 1/1 | Step 60/735 | Loss 0.0397
Epoch 1/1 | Step 70/735 | Loss 1.5947
Epoch 1/1 | Step 80/735 | Loss 0.0303


  0%|          | 0/1 [19:35<?, ?it/s]


KeyboardInterrupt: 

### EXTRACT NOTES

In [41]:
# single_embedding = classifier.forward(train_inputs[0:2], labels = train_labels[0:2], return_embeddings=True)

torch.Size([512])

In [14]:
# embeddings = classifier.forward(train_inputs, labels = train_labels, return_embeddings=True)
# embeddings_df = pd.DataFrame({'embedding': list(embeddings.detach().numpy())})
# df.reset_index(drop=True, inplace=True)
# df = pd.concat([df, embeddings_df], axis = 1)
# df.to_csv(mimic_iv_train_NOTE_EMBEDDINGS_path, index = False)

In [25]:
# embeddings_df = pd.DataFrame({'embedding': list(embeddings.detach().numpy())})

In [47]:
# df_small = df.iloc[0:5].copy().reset_index(drop=True)

In [48]:
# df_small = pd.concat([df_small, embeddings_df], axis = 1)

In [ ]:
Merge embeddings with dataframe

In [54]:
# embeddings[0].shape

torch.Size([768])

In [53]:
# df_small

,SUBJECT_ID,ICUSTAY_ID,t_start,t_end,NOTE_ID,text,delta,Capillary refill rate_1.0,Capillary refill rate_nan,Ethnicity_1.0,...,Age,Height,Weight,#past_IVs,t_from_last_IV_t_start,t_from_last_IV_t_end,INTIME,t_start_DT,time_since_note,embedding
0,10248673,33680639,24.000,24.388,10248673-DS-5,\nName: ___ Unit No: ___...,0,0,1,0,...,69.516622,168.0,66.0,1,18.250,11.167,2177-06-20 13:36:43,2177-06-21 13:36:43.000000000,901.611944,"[0.08366300067315613, -0.09525524751737473, 0...."
1,10248673,33680639,24.388,25.038,10248673-DS-5,\nName: ___ Unit No: ___...,0,0,1,0,...,69.516622,168.0,66.0,1,19.250,12.167,2177-06-20 13:36:43,2177-06-21 13:59:59.800000000,901.999944,"[0.09037796095801452, -0.1034607368093436, 0.2..."
2,10248673,33680639,25.038,25.388,10248673-DS-5,\nName: ___ Unit No: ___...,0,0,1,0,...,69.516622,168.0,66.0,1,19.900,12.817,2177-06-20 13:36:43,2177-06-21 14:38:59.800000000,902.649944,"[0.10461136858709813, -0.08170781207205698, 0...."
3,10248673,33680639,25.388,25.421,10248673-DS-5,\nName: ___ Unit No: ___...,0,0,1,0,...,69.516622,168.0,66.0,1,20.250,13.167,2177-06-20 13:36:43,2177-06-21 14:59:59.800000000,902.999944,"[0.09262458924580079, -0.1298178264939714, 0.2..."
4,10248673,33680639,25.421,26.388,10248673-DS-5,\nName: ___ Unit No: ___...,0,0,1,0,...,69.516622,168.0,66.0,1,20.283,13.200,2177-06-20 13:36:43,2177-06-21 15:01:58.600000000,903.032944,"[0.09782734270128297, -0.11514283711793492, 0...."


In [ ]:
# 

In [39]:
# outputs['classifier_last_hidden_state'][0].shape
# outputs['logits'][0]


tensor([0.0552], grad_fn=<SelectBackward0>)

In [184]:
# encoder_with_dense = T5EncoderWithDense(encoder = encoder, num_classes = 1)

In [186]:
# dense_layer_outputs = encoder_with_dense(inputs.input_ids)

In [188]:
# encoder_with_dense.hidden_states

[tensor([[0.0930]])]

In [187]:
# dense_layer_outputs

tensor([[0.0930]], grad_fn=<AddmmBackward0>)

In [178]:
dense(pooled_output)

tensor([[-0.0374]], grad_fn=<AddmmBackward0>)

In [44]:
# # Freeze the weights of the encoder layers
# for param in model_encoder_only.parameters():
#     param.requires_grad = False

In [ ]:
# from datasets import load_dataset
# # dataset = load_dataset('csv', data_files=mimic_iv_train_NOTE_path, split='train') # split = 'train
# df = pd.read_csv(mimic_iv_train_NOTE_path)[['NOTE_ID', 'text']]
# df.drop_duplicates(inplace=True)
# df.dropna(inplace=True) 
# df

In [136]:
# import torch
# inputs = tokenized_test_note
# outputs = tokenized_test_note
# outputs = model(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask, decoder_input_ids = inputs.input_ids)

In [138]:
# outputs.keys()

odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state'])

In [ ]:
# # import torch
# inputs = tokenized_test_note
# labels = torch.tensor([1]).unsqueeze(0)
# outputs = model_encoder_only

NameError: name 'test_note_to_encode' is not defined

In [90]:
# generated_outputs = []
# labels = torch.tensor([1]).unsqueeze(0)

# for i, row in notes_to_extract.iterrows():
#     input_text = row['text']
#     input_ids = tokenizer.encode(input_text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
#     outputs = model(**inputs, labels=labels)
#     generated_outputs.append(outputs)

NameError: name 'torch' is not defined

In [54]:
# import numpy as np

# # Generate a random target dataset with n = 1000
# n = 1000
# target_df = np.random.rand(n)